# Dewan Lab EPM Analysis
## 0: Run once to create all needed directories at beginning of a project

In [18]:
from dewan_calcium.helpers import DewanIOhandler
DewanIOhandler.create_project_framework('EPM')

## STEP 1: Always Execute! Load Libraries and User Settings
### STEP 1A: Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

from dewan_calcium import DewanDeconv
from dewan_calcium.helpers import DewanEPM, DewanIOhandler, DewanJSON

### STEP 1B: User Configurables

In [ ]:
animal = 'ANIMAL_GOES_HERE'
date = 'DATE_GOES_HERE'

# TODO: Get these configurables from the JSON file using the JSON Module

# Settings from Inscopix box
LED_power = 1
GAIN = 2.2
FOCUS = 250

fileHeader = animal + '-' + date + '-'

In [ ]:
test_path = Path('Z:\\2_Inscopix\\1_DTT\\3_EPM\VGLUT-20')

DewanIOhandler.create_project_framework('EPM', test_path)

## 2A: Import and pre-process the raw data

#### Copy DLC output .h5 file and labled video -> EPM_Analysis\RawData

In [ ]:
# New way that we should be grabbing the data

#STEP 2A.1: LOAD DLC DATA
# Read in data for processing.  Needs Cell Traces, Odor List, and GPIO file.
dlc_path_stem = Path(*['EPM_Analysis','DLC_Data'])
dlc_path = test_path.joinpath(dlc_path_stem)

labeled_video_path = list(dlc_path.glob('DLC*labeled*')) # Usually mp4 files, but this is more flexible
points_path = list(dlc_path.glob('*.h5'))

tracked_points = pd.read_hdf(points_path)  # Load tracked points
labeled_video = cv2.VideoCapture(str(labeled_video_path))  # Load Video

In [ ]:
#STEP 2A.2: LOAD INSCOPIX DATA

inscopix_path_stem = Path(*['InscopixProcessing', 'DataAnalysis'])  # Rename this to inscopix_path when done testing and delete the line below
inscopix_path = test_path.joinpath(inscopix_path_stem)

# We assume there is only one file that matches each query
cell_trace_path = list(inscopix_path.glob('*TRACES.csv'))[0]
GPIO_path = list(inscopix_path.glob('*GPIO.csv'))[0]
cell_props_path = list(inscopix_path.glob('*props.csv'))[0]
contours_path = list(inscopix_path.glob('*CONTOURS.json'))[0]
max_projection_path = list(inscopix_path.glob('*HD*MAX_PROJ.tiff'))[0]  # Match anything that includes HD and MAX_PROJ

In [ ]:
cell_trace_data = pd.read_csv(cell_trace_path, engine='pyarrow')
GPIO_data = pd.read_csv(GPIO_path, header=None, engine='pyarrow')
all_cell_props = pd.read_csv(cell_props_path, header=0, engine='pyarrow')
cell_keys, cell_outlines = DewanJSON.get_outline_coordinates(contours_path)

In [ ]:
# STEP 2A.2: PREPROCESSING

# STEP 2A.2.1: Drop the first row which contains all 'undecided' labels which is the Inscopix default label.
cell_trace_data.drop([0], inplace=True)

# STEP 2A.2.2: Force all dF/F values to be numbers and round times to 2 decimal places
cell_trace_data = cell_trace_data.apply(pd.to_numeric, errors='coerce')
# Set the times as the index so the listed data is all dF/F values
cell_trace_data[cell_trace_data.columns[0]] = cell_trace_data[cell_trace_data.columns[0]].round(2)
cell_trace_data.set_index(cell_trace_data.columns[0], inplace=True)

# STEP 2A.2.3: Remove spaces from column names
cell_trace_data.columns = [key.replace(' ', '') for key in cell_trace_data.columns.values]

# STEP 2A.2.4: REMOVE ALL MULTI-COMPONENT CELLS
# Generate a list of cell numbers based off the number of cells
cell_list = np.arange(len(all_cell_props['NumComponents'])) # Example Cell Numbers: 0, 1, 2, 3, 4
# Get indices where there are only one cell part
one_piece_cells = np.where(all_cell_props['NumComponents'] == 1)[0] # Example One-Component Indexes: 0, 1, 4
# Filter out all the multi-component cells, leaving only the one-piece cells
cell_list = cell_list[one_piece_cells] # Example Filtered Cell Numbers: 0, 1 ,4
cell_keys = cell_keys[one_piece_cells] # Example Filtered Cell Keys, C00, C01, C04
all_cell_props = all_cell_props.iloc[one_piece_cells] # Filter out two-piece cells as above

# STEP 2A.2.5: PARSE GPIO DATA
GPIO_data.iloc[:, 1] = GPIO_data.iloc[:, 1].str.replace(' ', '')  # Remove Random Spaces in Data
GPIO1 = np.array(GPIO_data.iloc[:, 1] == "GPIO-1")  # Get Sniff Sensor Data Truth Table
GPIO2 = np.array(GPIO_data.iloc[:, 1] == "GPIO-2")  # Get FV Actuation Data Truth Table
FV_data = np.array(GPIO_data.iloc[GPIO2,:]) # Create an array with FV values only

# STEP 2A.2.6: Make all numeric values floats and remove nullbytes

remove_null_bytes = lambda item: item.split('\x00')[0]
# For some reason the data will occasionally contain a very long string of null bytes '\\x00'
# this will remove everything after the null bytes,

# Iterate over each item and remove the nullbytes; simultaneously cast values to floats
FV_data[:, 0] = np.fromiter(map(remove_null_bytes, FV_data[:, 0]), 'float')
FV_data[:, 2] = np.fromiter(map(remove_null_bytes, FV_data[:, 2]), 'float')

cols = ['mouse_x', 'mouse_y', 'mouse_p', 'led_x', 'led_y', 'led_p'] 
# Reset the column names to something sensible
tracked_points.columns = cols 

In [ ]:
# There may be an instance where the model erroneously identified the LED for very short time periods
# find_led_start bins the possible LED on times (anywhere led_p > 0.98)
# We then find the bin with the largest size, which means it has the most frames where the LED is identified
# This is most likely the period where the experimenter turned on the LED
led_bins = np.array(DewanEPM.find_led_start(tracked_points))
true_led_bin = np.argmax(np.subtract(led_bins[:, 1], led_bins[:,0]))

led_on = led_bins[true_led_bin][0] # Find first row where the LED is 'on'
good_points = tracked_points.iloc[led_on:-1] # Delete all data before the LED is 'on'
good_points.reset_index(drop=True, inplace=True) # Reset the index

# Get X, Y coordinates, cast to int, and combine them into tuples
head_x = good_points['mouse_x'].astype(int)
head_y = good_points['mouse_y'].astype(int)
coordinates = list(zip(head_x, head_y))

In [ ]:

_ = labeled_video.set(cv2.CAP_PROP_POS_FRAMES, led_on)  # Set the first frame to the first LED frame
_, frame = labeled_video.read() # Read said frame

for pair in coordinates:  # Loop through each coordinate and draw a red point
    cv2.circle(frame, pair, 0, (0,0,255), -1)

# _ = cv2.imwrite(str(image_path), frame) # Save image